# Лабораторна робота № 5

## ІП-14 Бабіч Денис (09.07.2003)

---

# Підготовчий етап

## Імпортування необхідних модулів

In [1]:
import cv2
import numpy as np

## Створення допоміжних функцій

In [2]:
def load_image(relative_path: str) -> np.array:
    return cv2.imread(relative_path)

def show_image(image: np.array, title: str = "Preview") -> None:
    cv2.namedWindow(title, cv2.WINDOW_NORMAL)
    cv2.imshow(title, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def preprocess_image(image: np.ndarray, kernel_size: int) -> np.ndarray: 
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))

    preprocessed_frame = cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)
    preprocessed_frame = cv2.erode(preprocessed_frame, kernel, iterations = 1)
    preprocessed_frame = cv2.dilate(preprocessed_frame, kernel, iterations = 1)

    return preprocessed_frame

def apply_color_filtering_mask(image_stock: np.array, image_preprocessed: np.array, exclude_color_bottom_bound: tuple, exclude_color_upper_bound: tuple, roi_color: tuple = (0, 255, 0), roi_transparency: int = 60) -> tuple: 
    mask = cv2.bitwise_not(cv2.inRange(image_preprocessed, exclude_color_bottom_bound, exclude_color_upper_bound))

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    overlay = image_stock.copy()
    cv2.fillPoly(overlay, contours, roi_color)
    cv2.addWeighted(overlay, roi_transparency / 100, image_stock, 1 - roi_transparency / 100, 0, image_stock)

    return (image_stock, mask)

def overlay_objects_contours(image_stock: np.array, mask: np.array, width_min: int, height_min: int, width_max: int, height_max: int, roi_title: str, roi_color: tuple = (0, 255, 0), roi_thickness: int = 2) -> np.ndarray:
    ROI_TITLE_OFFSET = 10

    FONT_SCALE = 1.25

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        x, y, width, height = cv2.boundingRect(contour)

        if (width_min < width < width_max) and (height_min < height < height_max):
            cv2.rectangle(image_stock, (x, y), (x + width, y + height), roi_color, roi_thickness)
            cv2.putText(image_stock, roi_title, (x, y - ROI_TITLE_OFFSET), cv2.FONT_HERSHEY_COMPLEX_SMALL, FONT_SCALE, roi_color, roi_thickness)

    return image_stock

---

# Основний етап

## Зображення з високою роздільною здатністю

In [3]:
KERNEL_SIZE = 10

COLOR_BOTTOM_BOUND = (0, 0, 0)
COLOR_UPPER_BOUND = (255, 255, 50)

WIDTH_MIN = 10
HEIGHT_MIN = 10
WIDTH_MAX = 1000
HEIGHT_MAX = 1000

ROI_TITLE = "Island"

image = load_image("image_high_resolution.png")

preprocessed_image = preprocess_image(image, KERNEL_SIZE)

processed_image, mask = apply_color_filtering_mask(image, preprocessed_image, COLOR_BOTTOM_BOUND, COLOR_UPPER_BOUND)

processed_image = overlay_objects_contours(image, mask, WIDTH_MIN, HEIGHT_MIN, WIDTH_MAX, HEIGHT_MAX, ROI_TITLE)

show_image(processed_image)

## Зображення з низькою роздільною здатністю

In [4]:
KERNEL_SIZE = 5

COLOR_BOTTOM_BOUND = (0, 0, 0)
COLOR_UPPER_BOUND = (255, 255, 100)

WIDTH_MIN = 10
HEIGHT_MIN = 10
WIDTH_MAX = 1000
HEIGHT_MAX = 1000

ROI_TITLE = "Island"

image = load_image("image_low_resolution.png")

preprocessed_image = preprocess_image(image, KERNEL_SIZE)

processed_image, mask = apply_color_filtering_mask(image, preprocessed_image, COLOR_BOTTOM_BOUND, COLOR_UPPER_BOUND)

processed_image = overlay_objects_contours(image, mask, WIDTH_MIN, HEIGHT_MIN, WIDTH_MAX, HEIGHT_MAX, ROI_TITLE)

show_image(processed_image)